In [1]:
# import dependency
import requests
import json
import datetime

In [2]:
# some common data
amount = 10.50  # User A wants to transfer 10.50 dollars.
account_A = {
    'serverBalance': 5.50,  # Money in server account
    'username': '10055670',
    'password': 'aaa111'
}

account_B = {
    'serverBalance': 67.30,
    'username': '10055670',
    'password': 'aaa111'
}

account_Server = {
    'serverBalance': 50000000,
    'username': '10055670',
    'password': 'aaa111'
}

In [3]:
def login(username, password):
    url = 'https://hackathon.api.extnp.nab.com.au/v2/auth?v=1'
    headers = {
        'Content-Type': 'application/json',
        'x-nab-key': '0c12ce1a-f5a5-4933-b5aa-c27e14c757d7',
    }
    payload = {
        "loginRequest": {
            "brand": "nab",
            "lob": "nab",
            "credentials": {
                "apiStructType": "usernamePassword",
                "usernamePassword": {
                    "username": username,
                    "password": password
                }
            }
        }
    }
    response = requests.post(url, data=json.dumps(
        payload), headers=headers).json()
    return response


def generateToken(account):
    response = login(account['username'], account['password'])
    print(response['loginResponse']['tokens'][0]['value'])
    if response['status']['message'] is 'Success':
        print('Successfully log in')
    else:
        print('Error when try to log in')

    url = 'https://hackathon.api.extnp.nab.com.au/v2/token?v=1'
    headers = {
        'Content-Type': 'application/json',
        'x-nab-key': '0c12ce1a-f5a5-4933-b5aa-c27e14c757d7',
        'Authorization': str(response['loginResponse']['tokens'][0]['value'])
    }
    payload = {
        "tokenRequest": {
            "type": "accessToken"
        }
    }
    response = requests.post(url, data=json.dumps(
        payload), headers=headers).json()
    if response['status']['message'] is 'Success':
        print('Successfully generate a token')
    else:
        return response['status']['message']

    return response['tokenResponse']['token']


# User A requests to transfer money to User B
def user_request_transfer(account_A, account_B, account_name, tamount):
    # choose how to pay for money: via bank account or server account.
    if account_name is 'server':
        if tamount > account_A['serverBalance']:
            return 'You do not have enough balance. Please select another way to pay'
        else:
            account_A['serverBalance'] -= tamount
            account_B['serverBalance'] += tamount
    elif account_name is 'bank':
            # Try to make a payment
        return 'ggg'


# Transfer money between any two accounts
def transfer_X_Y(account_X, account_Y):
    return True


def get_lists_of_account(authorization_token):
    url = 'https://hackathon.api.extnp.nab.com.au/v2/accounts?v=1'
    headers = {
        'Content-Type': 'application/json',
        'x-nab-key': '0c12ce1a-f5a5-4933-b5aa-c27e14c757d7',
        'Authorization': str(authorization_token)
    }
    response = requests.get(url, headers=headers).json()
    return response

def make_payment(auth_One, auth_token_One, auth_token_Two, amount):
    url = 'https://hackathon.api.extnp.nab.com.au/v2/payment?v=1'
    headers = {
        'Content-Type': 'application/json',
        'x-nab-key': '0c12ce1a-f5a5-4933-b5aa-c27e14c757d7',
        'Authorization': '4132760c-8713-4c16-8f55-30ed765b699f'
    }

    data = {
        "paymentRequest": {
            "from": {
                "account": {
                    "apiStructType": "accountToken",
                    "accountToken": {
                        "token": "hbuYKpSndDdUtK3OCmqVhvFRgTk0wEKYRcXZl3m7Kr2QfpXyz_8Py9c06wh4VY13"
                    }
                }
            },
            "to": {
                "account": {
                    "apiStructType": "accountToken",
                    "accountToken": {
                        "token": "DLEmqT1YULGZSGfdyBHkCP_FuKW05bcA8ns3jJCWnZDoSxh-le10pYOUe4lI6uFf"
                    }
                }
            },
            "method": {
                "apiStructType": "transfer",
                "transfer": {
                    "amount": "500.00",
                    "statementReference": "Linked accounts",
                    "remitterName": "Mr. Bean"
                }
            },
            "recurrence": {
                "apiStructType": "onceOff",
                "onceOff": {
                    "paymentDate": "2012-12-12"
                }
            }
        }
    }

    response = requests.post(url, data=json.dumps(data), headers=headers).json()
    return response

def to_Virtual_Account(account_one, account_two, amount):
    account_two['serverBalance'] += amount
    account_one['serverBalance'] -= amount


In [4]:
def main():
    # init
    auth_A = login(account_A['username'], account_A['password'])
    account_lists_A = get_lists_of_account(auth_A['loginResponse']['tokens'][0]['value'])
    auth_B = login(account_B['username'], account_B['password'])
    account_lists_B = get_lists_of_account(auth_B['loginResponse']['tokens'][0]['value'])
    auth_Server = login(account_Server['username'], account_Server['password'])
    account_lists_Server = get_lists_of_account(auth_Server['loginResponse']['tokens'][0]['value'])
    
    auth_token_A = account_lists_A['accountsResponse']['accounts'][0]['accountToken']
    auth_token_B = account_lists_B['accountsResponse']['accounts'][0]['accountToken']
    auth_token_Server = account_lists_Server['accountsResponse']['accounts'][0]['accountToken']
    
    availableBalance_A = float(str(account_lists_A['accountsResponse']['accounts'][0]['availableBalance']))
    availableBalance_B = float(str(account_lists_B['accountsResponse']['accounts'][0]['availableBalance']))

    if availableBalance_A > amount:
        print('Enough Balance. Start transfering money')
        # firstly transfer money from A to Server's bank Account
        A_to_Server = make_payment(auth_A['loginResponse']['tokens'][0]['value'], auth_token_A, auth_token_Server, amount)
        print (A_to_Server)
        to_Virtual_Account(account_Server, account_B, amount)
    else:
        print('Not Enough Balance')
        
    print(account_A)
    print(account_B)
    print(account_Server)


if __name__ == '__main__':
    main()

Enough Balance. Start transfering money
{u'status': {u'message': u'Unauthorised', u'code': u'API-449'}}
{'username': '10055670', 'password': 'aaa111', 'serverBalance': 5.5}
{'username': '10055670', 'password': 'aaa111', 'serverBalance': 77.8}
{'username': '10055670', 'password': 'aaa111', 'serverBalance': 49999989.5}
